In [1]:
%config IPCompleter.use_jedi = False
%pdb off
# %load_ext viztracer
# from viztracer import VizTracer
%load_ext autoreload
%autoreload 3
import sys
from pathlib import Path
import neptune # for logging progress and results
from pyphoplacecellanalysis.General.Batch.NeptuneAiHelpers import set_environment_variables, neptune_output_figures
neptune_kwargs = {'project':"commander.pho/PhoDibaLongShort2023",
'api_token':"eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxOGIxODU2My1lZTNhLTQ2ZWMtOTkzNS02ZTRmNzM5YmNjNjIifQ=="}

# required to enable non-blocking interaction:
%gui qt5

from copy import deepcopy
from numba import jit
import numpy as np
import pandas as pd
from attrs import define, field, fields, Factory

# Pho's Formatting Preferences
# from pyphocorehelpers.preferences_helpers import set_pho_preferences, set_pho_preferences_concise, set_pho_preferences_verbose
# set_pho_preferences_concise()

## Pho's Custom Libraries:
from pyphocorehelpers.general_helpers import CodeConversion
from pyphocorehelpers.function_helpers import function_attributes
from pyphocorehelpers.print_helpers import print_keys_if_possible, print_value_overview_only, document_active_variables, objsize, print_object_memory_usage, debug_dump_object_member_shapes, TypePrintMode
from pyphocorehelpers.print_helpers import get_now_day_str, get_now_time_str, get_now_time_precise_str
from pyphocorehelpers.programming_helpers import IPythonHelpers
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager

# NeuroPy (Diba Lab Python Repo) Loading
# from neuropy import core
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.core.epoch import NamedTimerange, Epoch
from neuropy.core.ratemap import Ratemap
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.core.session.Formats.Specific.HiroDataSessionFormat import HiroDataSessionFormatRegisteredClass

## For computation parameters:
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.utils.dynamic_container import DynamicContainer
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths
from neuropy.core.neurons import NeuronType

from pyphocorehelpers.print_helpers import generate_html_string

# pyPhoPlaceCellAnalysis:
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import NeuropyPipeline # get_neuron_identities
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import export_pyqtgraph_plot, create_daily_programmatic_display_function_testing_folder_if_needed, session_context_to_relative_path
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_load_session, batch_extended_computations, SessionBatchProgress, batch_programmatic_figures, batch_extended_programmatic_figures
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap
from pyphoplacecellanalysis.Pho2D.PyQtPlots.Extensions.pyqtgraph_helpers import build_pyqtgraph_epoch_indicator_regions # Add session indicators to pyqtgraph plot
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import _helper_add_long_short_session_indicator_regions
import pyphoplacecellanalysis.External.pyqtgraph as pg

from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_perform_all_plots
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import JonathanFiringRateAnalysisResult
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import _find_any_context_neurons

# Plotting
import pylustrator
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import plot_multiple_raster_plot, plot_raster_plot
from pyphoplacecellanalysis.General.Mixins.DataSeriesColorHelpers import DataSeriesColorHelpers
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.helpers import _helper_make_scatterplot_clickable
from pyphoplacecellanalysis.General.Mixins.DataSeriesColorHelpers import UnitColoringMode, DataSeriesColorHelpers
from scripts.run_BatchAnalysis import post_compute_validate, _on_complete_success_execution_session
from PendingNotebookCode import PAPER_FIGURE_figure_1_add_replay_epoch_rasters, PAPER_FIGURE_figure_1_full


session_batch_status = {}
session_batch_errors = {}
enable_saving_to_disk = False

global_data_root_parent_path = find_first_extant_path([Path(r'W:\Data'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data'), Path(r'/home/halechr/turbo/Data')])
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"

Automatic pdb calling has been turned OFF
build_module_logger(module_name="Spike3D.pipeline"):
	 Module logger com.PhoHale.Spike3D.pipeline has file logging enabled and will log to EXTERNAL/TESTING/Logging/debug_com.PhoHale.Spike3D.pipeline.log


# Load Pipeline

In [2]:
%pdb off
# ==================================================================================================================== #
# Load Data                                                                                                            #
# ==================================================================================================================== #

active_data_mode_name = 'kdiba'

## Data must be pre-processed using the MATLAB script located here: 
#     neuropy/data_session_pre_processing_scripts/KDIBA/IIDataMat_Export_ToPython_2022_08_01.m
# From pre-computed .mat files:

local_session_root_parent_context = IdentifyingContext(format_name=active_data_mode_name) # , animal_name='', configuration_name='one', session_name=self.session_name
local_session_root_parent_path = global_data_root_parent_path.joinpath('KDIBA')

# [*] - indicates bad or session with a problem
# 0, 1, 2, 3, 4, 5, 6, 7, [8], [9], 10, 11, [12], 13, 14, [15], [16], 17, 
# curr_context: IdentifyingContext = good_contexts_list[1] # select the session from all of the good sessions here.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15') # DONE. Very good. Many good Pfs, many good replays.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43') # BAD
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31') # DONE, Good Pfs but no good replays
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6') # BAD
curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19') # DONE, GREAT, both good Pfs and replays!
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-13_15-22-3') # DONE, Good Pfs, no good epochs.
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-19_13-34-40') # BAD
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3') # BAD, Good Pfs strangely despite horrible map, no good epochs
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-12_15-25-59') # BAD, No Epochs
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_18-47-52')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-17_12-52-15')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_13-20-55')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_12-38-13')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_12-15-3')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_22-4-5')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54') # DONE


local_session_parent_path: Path = local_session_root_parent_path.joinpath(curr_context.animal, curr_context.exper_name) # 'gor01', 'one' - probably not needed anymore
basedir: Path = local_session_parent_path.joinpath(curr_context.session_name).resolve()
print(f'basedir: {str(basedir)}')

# Read if possible:
saving_mode = PipelineSavingScheme.SKIP_SAVING
force_reload = False

# Force write:
# saving_mode = PipelineSavingScheme.TEMP_THEN_OVERWRITE
# saving_mode = PipelineSavingScheme.OVERWRITE_IN_PLACE
# force_reload = True


## TODO: if loading is not possible, we need to change the `saving_mode` so that the new results are properly saved.


    # ==================================================================================================================== #
    # Load Pipeline                                                                                                        #
    # ==================================================================================================================== #
# with VizTracer(output_file=f"viztracer_{get_now_time_str()}-full_session_LOO_decoding_analysis.json", min_duration=200, tracer_entries=3000000, ignore_frozen=True) as tracer:
# epoch_name_includelist = ['maze']
epoch_name_includelist = None
active_computation_functions_name_includelist=[#'_perform_estimated_epochs_computation',  # AL:WAYS OFF
                                            '_perform_baseline_placefield_computation',
                                        # '_perform_time_dependent_placefield_computation', # AL:WAYS OFF
                                        '_perform_extended_statistics_computation',
                                        '_perform_position_decoding_computation', 
                                        '_perform_firing_rate_trends_computation',
                                        '_perform_pf_find_ratemap_peaks_computation',
                                        # '_perform_time_dependent_pf_sequential_surprise_computation' # AL:WAYS OFF
                                        '_perform_two_step_position_decoding_computation',
                                        # '_perform_recursive_latent_placefield_decoding' # AL:WAYS OFF
                                    ]
# active_computation_functions_name_includelist=['_perform_baseline_placefield_computation']

curr_active_pipeline = batch_load_session(global_data_root_parent_path, active_data_mode_name, basedir, epoch_name_includelist=epoch_name_includelist,
                                        computation_functions_name_includelist=active_computation_functions_name_includelist,
                                        saving_mode=saving_mode, force_reload=force_reload,
                                        skip_extended_batch_computations=True, debug_print=False, fail_on_exception=True) # , active_pickle_filename = 'loadedSessPickle_withParameters.pkl'


if not force_reload: # not just force_reload, needs to recompute whenever the computation fails.
    try:
        curr_active_pipeline.load_pickled_global_computation_results()
    except Exception as e:
        print(f'cannot load global results: {e}')

# 6m 1.1s
# 12m 15.6s


Automatic pdb calling has been turned OFF
basedir: /media/MAX/Data/KDIBA/gor01/two/2006-6-07_16-40-19
Loading loaded session pickle file results : /media/MAX/Data/KDIBA/gor01/two/2006-6-07_16-40-19/loadedSessPickle.pkl... done.
Loading pickled pipeline success: /media/MAX/Data/KDIBA/gor01/two/2006-6-07_16-40-19/loadedSessPickle.pkl.
property already present in pickled version. No need to save.
using provided computation_functions_name_includelist: ['_perform_baseline_placefield_computation', '_perform_extended_statistics_computation', '_perform_position_decoding_computation', '_perform_firing_rate_trends_computation', '_perform_pf_find_ratemap_peaks_computation', '_perform_two_step_position_decoding_computation']
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
updating computation_results...
done.
	 TODO: this will prevent recomputation even when the excludelist/includelist or comp

In [ ]:
%matplotlib qt
from neuropy.utils.matplotlib_helpers import interactive_select_grid_bin_bounds_2D
# fig, ax, rect_selector, set_extents = interactive_select_grid_bin_bounds_2D(curr_active_pipeline, epoch_name='maze', should_block_for_input=True)

grid_bin_bounds = interactive_select_grid_bin_bounds_2D(curr_active_pipeline, epoch_name='maze', should_block_for_input=True, should_apply_updates_to_pipeline=False)
print(f'grid_bin_bounds: {grid_bin_bounds}')
print(f"Add this to `specific_session_override_dict`:\n\n{curr_active_pipeline.get_session_context().get_initialization_code_string()}:dict(grid_bin_bounds=({(grid_bin_bounds[0], grid_bin_bounds[1]), (grid_bin_bounds[2], grid_bin_bounds[3])})),\n")

In [3]:
## Post Compute Validate 2023-05-16:
from scripts.run_BatchAnalysis import post_compute_validate
post_compute_validate(curr_active_pipeline)

curr_active_pipeline.reload_default_computation_functions()
extended_computations_include_includelist=['long_short_fr_indicies_analyses', 'jonathan_firing_rate_analysis', 'long_short_decoding_analyses', 'long_short_post_decoding'] # do only specifiedl , 'long_short_rate_remapping'
force_recompute_global = force_reload
# force_recompute_global = True
# extended_computations_include_includelist=['long_short_post_decoding'] # do only specifiedl
newly_computed_values = batch_extended_computations(curr_active_pipeline, include_includelist=extended_computations_include_includelist, include_global_functions=True, fail_on_exception=True, progress_print=True, force_recompute=force_recompute_global, debug_print=False)
if len(newly_computed_values) > 0:
	print(f'newly_computed_values: {newly_computed_values}. Saving global results...')
	try:
		# curr_active_pipeline.global_computation_results.persist_time = datetime.now()
		# Try to write out the global computation function results:
		curr_active_pipeline.save_global_computation_results()
	except Exception as e:
		print(f'\n\n!!WARNING!!: saving the global results threw the exception: {e}')
		print(f'\tthe global results are currently unsaved! proceed with caution and save as soon as you can!\n\n\n')
else:
	print(f'no changes in global results.')
# 10m 29.5s

were pipeline preprocessing parameters missing and updated?: False
included includelist is specified: ['long_short_fr_indicies_analyses', 'jonathan_firing_rate_analysis', 'long_short_decoding_analyses', 'long_short_post_decoding'], so only performing these extended computations.
Running batch_extended_computations(...) with global_epoch_name: "maze"
jonathan_firing_rate_analysis already computed.
long_short_fr_indicies_analyses already computed.
long_short_decoding_analyses already computed.
long_short_post_decoding already computed.
done with all batch_extended_computations(...).
no changes in global results.


In [ ]:
curr_active_pipeline.reload_default_display_functions()

neptuner = batch_perform_all_plots(curr_active_pipeline, enable_neptune=True)


### Pipeline Loading/Saving [Optional]

In [ ]:
curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.TEMP_THEN_OVERWRITE) # AttributeError: 'PfND_TimeDependent' object has no attribute '_included_thresh_neurons_indx'
# TypeError: cannot pickle 'MplMultiTab' object
# PicklingError: Can't pickle .set_closure_cell at 0x000002BF248F50D0>: it's not found as attr._compat.make_set_closure_cell..set_closure_cell
# TypeError: cannot pickle 'PyQt5.QtCore.pyqtSignal' object

In [ ]:
curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.OVERWRITE_IN_PLACE)

In [ ]:
curr_active_pipeline.save_global_computation_results() # PicklingError: Can't pickle .set_closure_cell at 0x000002BF248F50D0>: it's not found as attr._compat.make_set_closure_cell..set_closure_cell

In [ ]:
curr_active_pipeline.load_pickled_global_computation_results()

In [ ]:
curr_active_pipeline.clear_display_outputs()
curr_active_pipeline.clear_registered_output_files()

In [ ]:
curr_active_pipeline.save_global_computation_results()

# End Run

In [65]:
## long_short_decoding_analyses:
curr_long_short_decoding_analyses = curr_active_pipeline.global_computation_results.computed_data['long_short_leave_one_out_decoding_analysis']
## Extract variables from results object:
long_one_step_decoder_1D, short_one_step_decoder_1D, long_replays, short_replays, global_replays, long_shared_aclus_only_decoder, short_shared_aclus_only_decoder, shared_aclus, long_short_pf_neurons_diff, n_neurons, long_results_obj, short_results_obj, is_global = curr_long_short_decoding_analyses.long_decoder, curr_long_short_decoding_analyses.short_decoder, curr_long_short_decoding_analyses.long_replays, curr_long_short_decoding_analyses.short_replays, curr_long_short_decoding_analyses.global_replays, curr_long_short_decoding_analyses.long_shared_aclus_only_decoder, curr_long_short_decoding_analyses.short_shared_aclus_only_decoder, curr_long_short_decoding_analyses.shared_aclus, curr_long_short_decoding_analyses.long_short_pf_neurons_diff, curr_long_short_decoding_analyses.n_neurons, curr_long_short_decoding_analyses.long_results_obj, curr_long_short_decoding_analyses.short_results_obj, curr_long_short_decoding_analyses.is_global

# (long_one_step_decoder_1D, short_one_step_decoder_1D), (long_one_step_decoder_2D, short_one_step_decoder_2D) = compute_short_long_constrained_decoders(curr_active_pipeline, recalculate_anyway=True)
long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
long_epoch_context, short_epoch_context, global_epoch_context = [curr_active_pipeline.filtered_contexts[a_name] for a_name in (long_epoch_name, short_epoch_name, global_epoch_name)]
long_session, short_session, global_session = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_results, short_results, global_results = [curr_active_pipeline.computation_results[an_epoch_name]['computed_data'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_computation_config, short_computation_config, global_computation_config = [curr_active_pipeline.computation_results[an_epoch_name]['computation_config'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_pf1D, short_pf1D, global_pf1D = long_results.pf1D, short_results.pf1D, global_results.pf1D
long_pf2D, short_pf2D, global_pf2D = long_results.pf2D, short_results.pf2D, global_results.pf2D
decoding_time_bin_size = long_one_step_decoder_1D.time_bin_size # 1.0/30.0 # 0.03333333333333333

## Get global 'long_short_post_decoding' results:
curr_long_short_post_decoding = curr_active_pipeline.global_computation_results.computed_data['long_short_post_decoding']
expected_v_observed_result, curr_long_short_rr = curr_long_short_post_decoding.expected_v_observed_result, curr_long_short_post_decoding.rate_remapping
rate_remapping_df, high_remapping_cells_only = curr_long_short_rr.rr_df, curr_long_short_rr.high_only_rr_df
Flat_epoch_time_bins_mean, Flat_decoder_time_bin_centers, num_neurons, num_timebins_in_epoch, num_total_flat_timebins, is_short_track_epoch, is_long_track_epoch, short_short_diff, long_long_diff = expected_v_observed_result['Flat_epoch_time_bins_mean'], expected_v_observed_result['Flat_decoder_time_bin_centers'], expected_v_observed_result['num_neurons'], expected_v_observed_result['num_timebins_in_epoch'], expected_v_observed_result['num_total_flat_timebins'], expected_v_observed_result['is_short_track_epoch'], expected_v_observed_result['is_long_track_epoch'], expected_v_observed_result['short_short_diff'], expected_v_observed_result['long_long_diff']

jonathan_firing_rate_analysis_result = JonathanFiringRateAnalysisResult(**curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis.to_dict())
(epochs_df_L, epochs_df_S), (filter_epoch_spikes_df_L, filter_epoch_spikes_df_S), (good_example_epoch_indicies_L, good_example_epoch_indicies_S), (short_exclusive, long_exclusive, BOTH_subset, EITHER_subset, XOR_subset, NEITHER_subset), new_all_aclus_sort_indicies = PAPER_FIGURE_figure_1_add_replay_epoch_rasters(curr_active_pipeline)


good_example_epoch_indicies_L: [15 49]
good_example_epoch_indicies_S: [ 31  49  55  68  70  71  73  77  78  89  94 100 104 105 111 114 117 118
 122 123 131]


# ***NOW*** TODO 2023-06-20 22:01: - [ ] DONE FACTORING OUT `PAPER_FIGURE_figure_1_add_replay_epoch_rasters` and got plots to show up.
# - Just get both `plot_multiple_raster_plot` calls (long and short) to show up simultaneously

In [ ]:
## Stacked Epoch Plot
example_stacked_epoch_graphics = curr_active_pipeline.display('_display_long_and_short_stacked_epoch_slices', defer_render=False, save_figure=False)

In [ ]:
from pyphoplacecellanalysis.Pho2D.stacked_epoch_slices import DecodedEpochSlicesPaginatedFigureController

pagination_controller_L, pagination_controller_S = example_stacked_epoch_graphics.plot_data['controllers']
ax_L, ax_S = example_stacked_epoch_graphics.axes
final_figure_context_L, final_context_S = example_stacked_epoch_graphics.context

In [ ]:
## Capture current user selection
saved_selection_L = pagination_controller_L.save_selection()
saved_selection_S = pagination_controller_S.save_selection()


In [ ]:
from PendingNotebookCode import get_user_annotations

def update_selections_from_annotations(saved_selection, user_anootations, debug_print=True):
	""" 
	
	saved_selection_L.is_selected
	
	"""
	final_figure_context = saved_selection.figure_ctx
	was_annotation_found = False
	# try to find a matching user_annotation for the final_context_L
	for a_ctx, selections_array in user_anootations.items():
		an_item_diff = a_ctx.diff(final_figure_context)
		if debug_print:
			print(an_item_diff)
			print(f'\t{len(an_item_diff)}')
		if an_item_diff == {('user_annotation', 'selections')}:
			print(f'item found: {a_ctx}\nselections_array: {selections_array}')
			was_annotation_found = True
			saved_selection.is_selected = np.isin(saved_selection.flat_all_data_indicies, selections_array) # update the is_selected
			break # done looking
		
		# print(IdentifyingContext.subtract(a_ctx, final_context_L))
	if not was_annotation_found:
		print(f'WARNING: no matching context found in {len(user_anootations)} annotations. `saved_selection` will be returned unaltered.')
	return saved_selection

user_anootations = get_user_annotations()
user_anootations

In [ ]:
saved_selection_L = update_selections_from_annotations(saved_selection_L, user_anootations)
saved_selection_S = update_selections_from_annotations(saved_selection_S, user_anootations)
## re-apply the selections:
pagination_controller_L.restore_selections(saved_selection_L)
pagination_controller_S.restore_selections(saved_selection_S)


In [ ]:
pf1d_compare_graphics, (example_epoch_rasters_L, example_epoch_rasters_S), example_stacked_epoch_graphics = PAPER_FIGURE_figure_1_full(curr_active_pipeline) # did not display the pf1

In [5]:
# unit_colors_list = None # default rainbow of colors for the raster plots
neuron_qcolors_list = [pg.mkColor('black') for aclu in EITHER_subset.track_exclusive_aclus] # solid green for all
unit_colors_list = DataSeriesColorHelpers.qColorsList_to_NDarray(neuron_qcolors_list, is_255_array=True)

# Copy and modify the colors for the cells that are long/short exclusive:
unit_colors_list_L = deepcopy(unit_colors_list)
is_L_exclusive = np.isin(EITHER_subset.track_exclusive_aclus, long_exclusive.track_exclusive_aclus) # get long exclusive
unit_colors_list_L[0, is_L_exclusive] = 255 # [1.0, 0.0, 0.0, 1.0]
unit_colors_list_L[1, is_L_exclusive] = 0.0
unit_colors_list_L[2, is_L_exclusive] = 0.0

unit_colors_list_S = deepcopy(unit_colors_list)
is_S_exclusive = np.isin(EITHER_subset.track_exclusive_aclus, short_exclusive.track_exclusive_aclus) # get short exclusive
unit_colors_list_S[0, is_S_exclusive] = 0.0 # [1.0, 0.0, 0.0, 1.0]
unit_colors_list_S[1, is_S_exclusive] = 0.0
unit_colors_list_S[2, is_S_exclusive] = 255.0

In [6]:

# Common Tick Label
vtick = pg.QtGui.QPainterPath()

# Thicker Tick Label:
tick_width = 0.5
# tick_width = 10.0
half_tick_width = 0.5 * tick_width
vtick.moveTo(-half_tick_width, -0.5)
vtick.addRect(-half_tick_width, -0.5, tick_width, 1.0) # x, y, width, height

# pen = {'color': 'black', 'width': 1.0}
# pen = None
override_scatter_plot_kwargs = dict(pxMode=True, symbol=vtick, size=5) # , pen=pen

In [8]:
filter_epoch_spikes_df_L.spikes.rebuild_fragile_linear_neuron_IDXs()
filter_epoch_spikes_df_S.spikes.rebuild_fragile_linear_neuron_IDXs()

(                 t      t_seconds  t_rel_seconds  shank  cluster  aclu  qclu  \
 384691  30777371.0  526395.577744     945.480353      3       12    34     4   
 384696  30777725.0  526395.588619     945.491228      3       12    34     4   
 384702  30778144.0  526395.601491     945.504100     11        8    60     2   
 384703  30778244.0  526395.604563     945.507172     11        8    60     2   
 384710  30778545.0  526395.613809     945.516418      8       16    53     2   
 ...            ...            ...            ...    ...      ...   ...   ...   
 938255  82386208.0  527981.000405    2530.903014      1        7     6     2   
 938256  82386208.0  527981.000405    2530.903014      3       12    34     4   
 938261  82386358.0  527981.005013    2530.907622      8       16    53     2   
 938263  82386383.0  527981.005781    2530.908390      3       12    34     4   
 938268  82386721.0  527981.016164    2530.918773      3       12    34     4   
 
                  x       

In [9]:
# Test single `plot_raster_plot` calls
an_epoch = list(epochs_df_L.itertuples())[0]
an_epoch_spikes_df = filter_epoch_spikes_df_L[filter_epoch_spikes_df_L['replay_epoch_id'] == an_epoch.Index]

_out_single_raster_plot = plot_raster_plot(an_epoch_spikes_df, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=None, unit_colors_list=unit_colors_list, scatter_app_name="test1")
_out_single_raster_plot2 = plot_raster_plot(an_epoch_spikes_df, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=new_all_aclus_sort_indicies, unit_colors_list=unit_colors_list, scatter_app_name="test2")

libEGL warning: egl: failed to create dri2 screen


In [10]:
app_L, win_L, plots_L, plots_data_L = plot_multiple_raster_plot(epochs_df_L, filter_epoch_spikes_df_L, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=new_all_aclus_sort_indicies, unit_colors_list=unit_colors_list, scatter_plot_kwargs=override_scatter_plot_kwargs,
										epoch_id_key_name='replay_epoch_id', scatter_app_name="Long Decoded Example Replays")

In [ ]:
plot_item = list(plots_L.ax.values())[0]
plot_item.showGrid(False, False, 100.0)

In [ ]:
#TODO 2023-06-23 07:52: - [ ] This GridItem method seems to actually work unlike the other showGrid methods and stuff.
# create a GridItem and add it to the plot
grid = pg.GridItem()
plot_item.addItem(grid)
 # set the properties of the grid
grid.setTickSpacing([None], [1])
grid.setPen(pg.mkPen('#888888', width=1))
grid.setZValue(-100)

In [11]:
app_S, win_S, plots_S, plots_data_S = plot_multiple_raster_plot(epochs_df_S, filter_epoch_spikes_df_S, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=new_all_aclus_sort_indicies, unit_colors_list=unit_colors_list, scatter_plot_kwargs=override_scatter_plot_kwargs,
                                                                 epoch_id_key_name='replay_epoch_id', scatter_app_name="Short Decoded Example Replays")

In [ ]:
app_alt, win_alt, plots_alt, plots_data_alt = plot_multiple_raster_plot(epochs_df_L, filter_epoch_spikes_df_L, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=None, unit_colors_list=unit_colors_list, scatter_plot_kwargs=override_scatter_plot_kwargs,
                                                                         epoch_id_key_name='replay_epoch_id', scatter_app_name="ALT Long Decoded Example Replays")

### Testing `plot_kourosh_activity_style_figure` for debugging:

In [13]:
from pyphoplacecellanalysis.Analysis.Decoder.decoder_result import plot_kourosh_activity_style_figure

# plot_aclus = EITHER_subset.track_exclusive_aclus.copy()
plot_aclus = EITHER_subset.track_exclusive_aclus[new_all_aclus_sort_indicies].copy()
_out_A = plot_kourosh_activity_style_figure(long_results_obj, long_session, plot_aclus, epoch_idx=15, callout_epoch_IDXs=np.arange(5), skip_rendering_callouts=False)

active_epoch_n_timebins = 5
len(shared_aclus) = 39
callout_flat_timebin_IDXs = array([86, 87, 88, 89, 90])
plots_data.callout_time_bins: [array([572.24842637, 572.28142637, 572.31442637, 572.34742637,
       572.38042637]), array([572.28142637, 572.31442637, 572.34742637, 572.38042637,
       572.41342637])]
start_ts: [572.24842637 572.28142637 572.31442637 572.34742637 572.38042637], end_ts: [572.28142637 572.31442637 572.34742637 572.38042637 572.41342637]
setting region[86]: 572.2484263688093, 572.2814263688085 :: end_t - start_t = 0.032999999999219654
setting region[87]: 572.2814263688085, 572.3144263688077 :: end_t - start_t = 0.032999999999219654
setting region[88]: 572.3144263688077, 572.347426368807 :: end_t - start_t = 0.032999999999219654
setting region[89]: 572.347426368807, 572.3804263688062 :: end_t - start_t = 0.032999999999219654
setting region[90]: 572.3804263688062, 572.4134263688054 :: end_t - start_t = 0.032999999999219654


In [ ]:
_out_n = plot_kourosh_activity_style_figure(long_results_obj, long_session, EITHER_subset.track_exclusive_aclus, epoch_idx=49, callout_epoch_IDXs=np.arange(6), skip_rendering_callouts=False)

# Figure 2) Firing Rate Bar Graphs

In [23]:
neuron_replay_stats_df = jonathan_firing_rate_analysis_result.neuron_replay_stats_df
neuron_replay_stats_df # for `neuron_replay_stats_df` we have long/short_non_replay_mean but not specifically the laps mean. Actually we can just get that from the peak firing rates from pf1D yeah?


long_pf1D.ratemap.tuning_curve_unsmoothed_peak_firing_rates

# long_exclusive.track_exclusive_df # has 'long_non_replay_mean', 'short_non_replay_mean', 'short_replay_mean', etc...

TrackExclusivePartitionSubset(is_aclu_pf_track_exclusive=2     False
3     False
4     False
5     False
6     False
      ...  
61    False
62    False
63    False
64    False
65    False
Length: 64, dtype: bool, track_exclusive_aclus=array([12, 21, 48]), track_exclusive_df=    long_pf_peak_x  has_long_pf  short_pf_peak_x  has_short_pf  has_na  \
12       23.397021         True              NaN         False    True   
21      227.397021         True              NaN         False    True   
48      217.397021         True              NaN         False    True   

                      track_membership  long_non_replay_mean  \
12  SplitPartitionMembership.LEFT_ONLY              0.391955   
21  SplitPartitionMembership.LEFT_ONLY              1.856750   
48  SplitPartitionMembership.LEFT_ONLY              0.456971   

    short_non_replay_mean  non_replay_diff  long_replay_mean  \
12               0.256217        -0.135738          8.701443   
21               1.552593        -0.304158

In [32]:
# Reviewing Instanenous firing rates:
active_firing_rate_trends = curr_active_pipeline.computation_results[global_epoch_name].computed_data.get('firing_rate_trends', None)
active_time_bin_size_seconds = active_firing_rate_trends['time_bin_size_seconds']
active_all_session_spikes = active_firing_rate_trends['all_session_spikes']
pf_included_spikes_only = active_firing_rate_trends['pf_included_spikes_only']
active_time_binning_container, active_time_window_edges, active_time_window_edges_binning_info, active_time_binned_unit_specific_binned_spike_rate, active_time_binned_unit_specific_binned_spike_counts = pf_included_spikes_only['time_binning_container'], pf_included_spikes_only['time_window_edges'], pf_included_spikes_only['time_window_edges_binning_info'], pf_included_spikes_only['time_binned_unit_specific_binned_spike_rate'], pf_included_spikes_only['time_binned_unit_specific_binned_spike_counts']

# active_time_binning_container = pf_included_spikes_only['time_binning_container']
# active_time_window_edges = pf_included_spikes_only['time_window_edges']
# active_time_window_edges_binning_info = pf_included_spikes_only['time_window_edges_binning_info']
# active_time_binned_unit_specific_binned_spike_rate = pf_included_spikes_only['time_binned_unit_specific_binned_spike_rate']
# active_time_binned_unit_specific_binned_spike_counts = pf_included_spikes_only['time_binned_unit_specific_binned_spike_counts']


In [ ]:
active_all_session_spikes.

In [135]:
# Instantaneous versions:
from quantities import ms, s, Hz
from neo.core.spiketrain import SpikeTrain
from elephant.kernels import GaussianKernel
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import _epoch_unit_avg_firing_rates

instantaneous_time_bin_size_seconds = 0.02 # 20ms
# sess_unit_specific_inst_spike_rate_values_df, sess_unit_specific_inst_spike_rate, sess_unit_split_spiketrains = SpikeRateTrends.compute_instantaneous_time_firing_rates(active_session_spikes_df, time_bin_size_seconds=time_bin_size_seconds, t_start=computation_result.sess.t_start, t_stop=computation_result.sess.t_stop)


In [127]:
# epochs_df_L, filter_epoch_spikes_df_L, included_neuron_ids=EITHER_subset.track_exclusive_aclus
print(f'{EITHER_subset.track_exclusive_aclus}')
n_cells = len(EITHER_subset.track_exclusive_aclus)
epoch_inst_fr_df_list, epoch_inst_fr_signal_list, epoch_agg_firing_rates_list = SpikeRateTrends.compute_epochs_unit_avg_inst_firing_rates(spikes_df=filter_epoch_spikes_df_L, filter_epochs=epochs_df_L, included_neuron_ids=EITHER_subset.track_exclusive_aclus, debug_print=True)

[ 5  6  7  8 11 12 15 17 18 19 20 21 24 25 26 27 28 31 34 35 39 40 41 43
 44 45 48 49 50 51 52 53 55 56 60 62 63 64 65]
filter_epochs: 2


In [110]:
epoch_inst_fr_df_list, epoch_inst_fr_signal_list, epoch_agg_firing_rates_list = SpikeRateTrends.compute_epochs_unit_avg_inst_firing_rates(spikes_df=filter_epoch_spikes_df_S, filter_epochs=epochs_df_S, included_neuron_ids=EITHER_subset.track_exclusive_aclus, debug_print=True)

filter_epochs: 21


In [129]:
n_epochs = len(epoch_inst_fr_df_list)

In [130]:
inst_fr_trends = SpikeRateTrends(inst_fr_df_list=epoch_inst_fr_df_list, inst_fr_signals_list=epoch_inst_fr_signal_list)
inst_fr_trends

SpikeRateTrends(inst_fr_df_list=[    5          6             7    8          11   12   15   17   18   19  ...  \
0  0.0   0.000000  2.419707e+01  0.0   0.000074  0.0  0.0  0.0  0.0  0.0  ...   
1  0.0   0.000223  3.989423e+01  0.0   0.006840  0.0  0.0  0.0  0.0  0.0  ...   
2  0.0   0.020075  2.419707e+01  0.0   0.234975  0.0  0.0  0.0  0.0  0.0  ...   
3  0.0   0.664777  5.399097e+00  0.0   3.142733  0.0  0.0  0.0  0.0  0.0  ...   
4  0.0   8.098645  4.431848e-01  0.0  17.497633  0.0  0.0  0.0  0.0  0.0  ...   
5  0.0  36.295609  1.338302e-02  0.0  44.144186  0.0  0.0  0.0  0.0  0.0  ...   
6  0.0  59.841342  1.486720e-04  0.0  51.992764  0.0  0.0  0.0  0.0  0.0  ...   
7  0.0  36.295609  0.000000e+00  0.0  26.896621  0.0  0.0  0.0  0.0  0.0  ...   
8  0.0   8.098645  1.421085e-15  0.0   5.620689  0.0  0.0  0.0  0.0  0.0  ...   

    51   52   53         55   56   60   62         63            64   65  
0  0.0  0.0  0.0   0.000074  0.0  0.0  0.0  14.798085  8.526513e-15  0.0  
1  0.0

In [136]:
inst_fr_trends = SpikeRateTrends.init_from_spikes_and_epochs(spikes_df=filter_epoch_spikes_df_S, filter_epochs=epochs_df_S, included_neuron_ids=EITHER_subset.track_exclusive_aclus)
# inst_fr_trends

SpikeRateTrends(inst_fr_df_list=[    5    6          7    8          11   12   15            17   18   19  ...  \
0  0.0  0.0   2.927832  0.0   0.000074  0.0  0.0  1.749763e+01  0.0  0.0  ...   
1  0.0  0.0  15.019677  0.0   0.006692  0.0  0.0  4.414419e+01  0.0  0.0  ...   
2  0.0  0.0  34.745199  0.0   0.221592  0.0  0.0  5.199276e+01  0.0  0.0  ...   
3  0.0  0.0  44.144186  0.0   2.699548  0.0  0.0  2.689662e+01  0.0  0.0  ...   
4  0.0  0.0  34.745273  0.0  12.098536  0.0  0.0  5.620689e+00  0.0  0.0  ...   
5  0.0  0.0  15.026368  0.0  19.947114  0.0  0.0  4.498764e-01  0.0  0.0  ...   
6  0.0  0.0   3.149425  0.0  12.098536  0.0  0.0  1.345736e-02  0.0  0.0  ...   
7  0.0  0.0   2.927907  0.0   2.699548  0.0  0.0  1.486720e-04  0.0  0.0  ...   
8  0.0  0.0  12.105302  0.0   0.221592  0.0  0.0  2.486900e-15  0.0  0.0  ...   
9  0.0  0.0  19.947188  0.0   0.006692  0.0  0.0  4.973799e-15  0.0  0.0  ...   

             51            52            53         55   56            60  

In [131]:
epoch_agg_firing_rates_list = np.vstack([a_signal.max(axis=0) for a_signal in inst_fr_trends.inst_fr_signals_list]) # find the peak within each epoch (for all cells) using `.max(...)`
assert epoch_agg_firing_rates_list.shape == (n_epochs, n_cells)
epoch_agg_firing_rates_list # .shape (n_epochs, n_cells)
cell_agg_firing_rates_list = epoch_agg_firing_rates_list.max(axis=0) # find the peak over all epochs (for all cells) using `.max(...)`
assert cell_agg_firing_rates_list.shape == (n_cells,)
cell_agg_firing_rates_list # .shape (n_cells,)


In [111]:
epoch_inst_fr_df_list[0]

,5,6,7,8,11,12,15,17,18,19,...,51,52,53,55,56,60,62,63,64,65
0,0.0,0.0,2.927832,0.0,0.000074,0.0,0.0,1.749763e+01,0.0,0.0,...,2.131628e-15,2.486900e-15,1.749763e+01,0.006766,0.0,4.883733e+01,0.0,0.0,0.000074,0.000074
1,0.0,0.0,15.019677,0.0,0.006692,0.0,0.0,4.414419e+01,0.0,0.0,...,0.000000e+00,7.105427e-16,4.414419e+01,0.228284,0.0,8.518755e+01,0.0,0.0,0.006692,0.006692
2,0.0,0.0,34.745199,0.0,0.221592,0.0,0.0,5.199276e+01,0.0,0.0,...,1.486720e-04,0.000000e+00,5.199276e+01,2.921141,0.0,7.259122e+01,0.0,0.0,0.221592,0.221592
3,0.0,0.0,44.144186,0.0,2.699548,0.0,0.0,2.689662e+01,0.0,0.0,...,1.345736e-02,7.433598e-05,2.689662e+01,14.798085,0.0,5.069242e+01,0.0,0.0,2.699548,2.699548
4,0.0,0.0,34.745273,0.0,12.098536,0.0,0.0,5.620689e+00,0.0,0.0,...,4.498764e-01,6.765847e-03,5.620689e+00,32.045650,0.0,2.508344e+01,0.0,0.0,12.098536,12.098536
5,0.0,0.0,15.026368,0.0,19.947114,0.0,0.0,4.498764e-01,0.0,0.0,...,5.620689e+00,2.282839e-01,4.498764e-01,32.045650,0.0,5.425863e+00,0.0,0.0,19.947114,19.947114
6,0.0,0.0,3.149425,0.0,12.098536,0.0,0.0,1.345736e-02,0.0,0.0,...,2.689662e+01,2.921141e+00,1.345736e-02,14.798085,0.0,4.434822e-01,0.0,0.0,12.098536,12.098536
7,0.0,0.0,2.927907,0.0,2.699548,0.0,0.0,1.486720e-04,0.0,0.0,...,5.199276e+01,1.479808e+01,1.486720e-04,2.921141,0.0,1.338302e-02,0.0,0.0,2.699548,2.699548
8,0.0,0.0,12.105302,0.0,0.221592,0.0,0.0,2.486900e-15,0.0,0.0,...,4.414419e+01,3.204565e+01,2.486900e-15,0.228284,0.0,1.486720e-04,0.0,0.0,0.221592,0.221592
9,0.0,0.0,19.947188,0.0,0.006692,0.0,0.0,4.973799e-15,0.0,0.0,...,1.749763e+01,3.204565e+01,4.973799e-15,0.006766,0.0,5.684342e-15,0.0,0.0,0.006692,0.006692


In [92]:
epoch_inst_fr_df_list[1]

,5,6,7,8,11,12,15,17,18,19,...,51,52,53,55,56,60,62,63,64,65
0,0.0,3.402620,5.092573,1.209854,1.666123,0.0,1.209854,4.570997,1.760327,0.0,...,1.984763,3.989423,6.019455,0.0,0.0,13.280653,0.0,0.0,5.416257,0.0
1,0.0,3.573062,5.362004,1.330426,1.760327,0.0,1.330426,4.893516,1.841351,0.0,...,1.994711,3.969525,6.454785,0.0,0.0,13.752779,0.0,0.0,5.869637,0.0
2,0.0,3.715540,5.589640,1.448458,1.841351,0.0,1.448458,5.186776,1.906939,0.0,...,1.984763,3.910427,6.852899,0.0,0.0,14.125031,0.0,0.0,6.299536,0.0
3,0.0,3.826113,5.769092,1.561270,1.906939,0.0,1.561270,5.443028,1.955213,0.0,...,1.955213,3.813878,7.203355,0.0,0.0,14.388797,0.0,0.0,6.695601,0.0
4,0.0,3.901650,5.895190,1.666123,1.955213,0.0,1.666123,5.655229,1.984763,0.0,...,1.906939,3.682701,7.496580,0.0,0.0,14.537886,0.0,0.0,7.047786,0.0
5,0.0,3.939976,5.964237,1.760327,1.984763,0.0,1.760327,5.817366,1.994711,0.0,...,1.841351,3.520653,7.724305,0.0,0.0,14.568757,0.0,0.0,7.346767,0.0
6,0.0,3.939976,5.974186,1.841351,1.994711,0.0,1.841351,5.924739,1.984763,0.0,...,1.760327,3.332246,7.879952,0.0,0.0,14.480656,0.0,0.0,7.584352,0.0
7,0.0,3.901650,5.924739,1.906939,1.984763,0.0,1.906939,5.974186,1.955213,0.0,...,1.666123,3.122539,7.958948,0.0,0.0,14.275639,0.0,0.0,7.753854,0.0
8,0.0,3.826113,5.817366,1.955213,1.955213,0.0,1.955213,5.964237,1.906939,0.0,...,1.561270,2.896916,7.958948,0.0,0.0,13.958506,0.0,0.0,7.850403,0.0
9,0.0,3.715540,5.655229,1.984763,1.906939,0.0,1.984763,5.895190,1.841351,0.0,...,1.448458,2.660852,7.879952,0.0,0.0,13.536614,0.0,0.0,7.871176,0.0


In [98]:
np.vstack(epoch_agg_firing_rates_list).shape # (n_epochs, n_cells) (2, 39)


(2, 39)

In [104]:
epoch_agg_firing_rates_list.shape

(2, 39)

In [ ]:
# epoch_inst_fr_df_list

epoch_agg_firing_rates_list[0].shape
epoch_agg_firing_rates_list[1].shape

# np.vstack(epoch_avg_firing_rates_list)

In [37]:
active_all_session_spikes.instantaneous_unit_specific_spike_rate_values_df # (n_time_bins, n_cells) 5175 rows × 64 columns

,2,3,4,5,6,7,8,9,10,11,...,56,57,58,59,60,61,62,63,64,65
0,4.164765,0.000000e+00,42.765459,8.765637e-02,0.000007,7.978883,3.989438e+00,22.555398,0.0,0.000000e+00,...,0.000000e+00,4.778271,0.262954,12.406521,3.989445,0.000000e+00,0.175298,1.994711e+00,3.989423,1.752830e-01
1,4.865853,5.263280e-16,23.014634,2.169994e+00,0.087642,0.788788,3.505660e-01,15.891130,0.0,0.000000e+00,...,0.000000e+00,19.004123,6.334700,11.025337,0.438208,9.868649e-16,4.164706,8.764150e-02,0.175305,3.989430e+00
2,16.308302,0.000000e+00,29.591438,4.077064e+00,1.994711,10.148877,3.989445e+00,22.993680,0.0,3.508853e-16,...,2.105312e-15,20.998872,8.241778,13.370644,5.984149,0.000000e+00,4.164721,7.433598e-06,0.262947,2.629319e-01
3,5.041143,1.315820e-16,20.757072,1.752904e-01,0.087642,4.515272,2.629245e-01,11.814110,0.0,7.017706e-16,...,0.000000e+00,7.386465,0.438230,23.256485,0.262925,3.947460e-16,0.350581,7.017706e-16,6.071776,2.082368e+00
4,8.241852,3.837808e-16,37.482516,2.230079e-05,0.000007,2.170032,1.994726e+00,20.735999,0.0,7.017706e-16,...,7.017706e-16,12.231275,1.994741,18.916533,0.000022,4.166763e-16,3.989445,0.000000e+00,2.257651,2.082353e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5170,6.860616,9.577981e-16,32.462431,1.208927e-16,0.175283,4.515294,4.252355e+00,11.551193,0.0,2.629394e-01,...,2.170002e+00,26.127730,4.865920,55.894191,0.087642,8.766380e-02,0.175290,2.194045e-16,0.000052,0.000000e+00
5171,8.417105,2.230079e-05,42.085451,0.000000e+00,1.994719,12.318834,4.164713e+00,24.637758,0.0,6.159417e+00,...,1.752979e-01,17.535439,0.263051,30.906098,1.994711,2.257643e+00,0.000037,2.230079e-05,0.000007,0.000000e+00
5172,4.427660,2.629245e-01,23.102372,0.000000e+00,0.087649,4.515287,1.752979e-01,7.561756,0.0,4.252362e+00,...,1.994726e+00,13.545934,2.608232,18.236601,0.087642,6.159417e+00,0.262925,2.629245e-01,0.087642,1.003313e-15
5173,2.257666,5.984134e+00,33.405437,0.000000e+00,0.087649,0.175328,1.486720e-05,12.669490,0.0,3.505883e-01,...,1.752830e-01,20.823589,14.225904,25.075980,0.000007,2.257643e+00,5.984134,5.984134e+00,1.994711,9.452449e-16


In [27]:
print(f'long_exclusive.track_exclusive_aclus: {long_exclusive.track_exclusive_aclus}')
long_pf1D.ratemap.tuning_curve_unsmoothed_peak_firing_rates[np.isin(long_pf1D.cell_ids, long_exclusive.track_exclusive_aclus)]

long_exclusive.track_exclusive_aclus: [12 21 48]


array([14.98488766,  4.03439283, 22.47733149])

In [30]:
long_exclusive.track_exclusive_df[['long_non_replay_mean', 'long_replay_mean', 'long_mean']]

,long_non_replay_mean,long_replay_mean,long_mean
12,0.391955,8.701443,4.546699
21,1.856750,16.874124,9.365437
48,0.456971,8.140116,4.298543


In [25]:
## If they're LxC's they definitionally don't have a placefield on the other track, so we'll need to use the other data.
short_pf1D.ratemap.tuning_curve_unsmoothed_peak_firing_rates[np.isin(short_pf1D.cell_ids, short_exclusive.track_exclusive_aclus)]

array([ 9.63314632, 25.61054657,  9.03199107])

# Prepare Figure 3) `_display_short_long_firing_rate_index_comparison` for debugging:

In [14]:
import matplotlib as mpl
import matplotlib.pyplot as plt
# from pyphoplacecellanalysis.General.Mixins.ExportHelpers import create_daily_programmatic_display_function_testing_folder_if_needed, session_context_to_relative_path, programmatic_display_to_PDF
# from pyphoplacecellanalysis.General.Mixins.ExportHelpers import build_pdf_metadata_from_display_context # newer version of build_pdf_export_metadata
from neuropy.core.neuron_identities import PlotStringBrevityModeEnum
from neuropy.plotting.ratemaps import BackgroundRenderingOptions
_bak_rcParams = mpl.rcParams.copy()
# mpl.rcParams['toolbar'] = 'None' # disable toolbars
# matplotlib.use('AGG') # non-interactive backend ## 2022-08-16 - Surprisingly this works to make the matplotlib figures render only to .png file, not appear on the screen!
%matplotlib qt

## Activate pylustrator
pylustrator.start()


In [15]:
common_kwargs = dict(defer_render=False, save_figure=True)


In [16]:
_out = curr_active_pipeline.display('_display_short_long_firing_rate_index_comparison', curr_active_pipeline.get_session_context(), **common_kwargs)

qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


	 saved /home/halechr/repo/Spike3D/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/2023-06-23/kdiba_gor01_two_2006-6-07_16-40-19_long_short_firing_rate_indicies_display_long_short_laps.png


In [ ]:
plt.show()

In [ ]:
from adjustText import adjust_text # `pip install adjustText` for dynamically positioning text labels to prevent overlap
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import _plot_session_long_short_track_firing_rate_figures

curr_active_pipeline.reload_default_display_functions()


In [ ]:
curr_active_pipeline.get_session_context()

In [17]:
_out2 = curr_active_pipeline.display('_display_long_and_short_firing_rate_replays_v_laps', curr_active_pipeline.get_session_context(), **common_kwargs)

qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


	 saved /home/halechr/repo/Spike3D/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/2023-06-23/kdiba_gor01_two_2006-6-07_16-40-19_maze1_plot_single_track_firing_rate_compare.png
	 saved /home/halechr/repo/Spike3D/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/2023-06-23/kdiba_gor01_two_2006-6-07_16-40-19_maze2_plot_single_track_firing_rate_compare.png


In [ ]:
_out2.figures[0]

# 2023-05-19 - Testing S-only emergence, L-only replays in S, peak position remappings 

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import _prepare_spikes_df_from_filter_epochs, _find_example_epochs
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import JonathanFiringRateAnalysisResult
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import SplitPartitionMembership, SetPartition
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import TrackExclusivePartitionSubset
# approach copied from `pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations._epoch_unit_avg_firing_rates`


In [ ]:
""" 24 cells, 133 Epochs, 789 Total Timebins
# Other
	
	- is_non_firing_time_bin: numpy.ndarray - (24, 789)
	- all_epochs_num_epoch_time_bins: numpy.ndarray - (133,)
	
    ## Indexing Helpers:
		- all_epochs_reverse_flat_epoch_indicies_array: numpy.ndarray - (789,)
		- split_by_epoch_reverse_flattened_time_bin_indicies: list - (133,)
    
	- original_1D_decoder: pyphoplacecellanalysis.Analysis.Decoder.reconstruction.BasePositionDecoder
		- pf: neuropy.analyses.placefields.PfND
		- neuron_IDXs: numpy.ndarray - (24,)
		- neuron_IDs: numpy.ndarray - (24,)
		- F: numpy.ndarray - (119, 24)
		- P_x: numpy.ndarray - (119, 1)
        
	- flat_all_epochs_decoded_epoch_time_bins: numpy.ndarray - (24, 789)

# Measured
	- flat_all_epochs_measured_cell_spike_counts: numpy.ndarray - (24, 789)
	- flat_all_epochs_measured_cell_firing_rates: numpy.ndarray - (24, 789)


# Expected
	- flat_all_epochs_computed_expected_cell_firing_rates: numpy.ndarray - (24, 789)
	- flat_all_epochs_difference_from_expected_cell_spike_counts: numpy.ndarray - (24, 789)
	- flat_all_epochs_difference_from_expected_cell_firing_rates: numpy.ndarray - (24, 789)
    
    
    
## Epoch-based
	- all_epochs_decoded_epoch_time_bins_mean: numpy.ndarray - (133, 24)

    
    
    
- flat_all_epochs_measured_cell_firing_rates: numpy.ndarray - .shape: (24, 789) 
- flat_all_epochs_computed_expected_cell_firing_rates: numpy.ndarray - .shape: (24, 789)


"""

# 2023-04-13 - Find Good looking epochs:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib qt
import matplotlib.patheffects as path_effects

from pyphoplacecellanalysis.Pho2D.stacked_epoch_slices import DecodedEpochSlicesPaginatedFigureController
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import DecodedFilterEpochsResult
from pyphoplacecellanalysis.Analysis.Decoder.decoder_result import LeaveOneOutDecodingAnalysisResult


In [ ]:
_out_pagination_controller_L = DecodedEpochSlicesPaginatedFigureController.init_from_decoder_data(long_results_obj.active_filter_epochs, long_results_obj.all_included_filter_epochs_decoder_result, included_epoch_indicies=good_epoch_indicies_L, 
	xbin=long_results_obj.original_1D_decoder.xbin, global_pos_df=global_session.position.df, a_name='Long SubsetEpochs', active_context=long_epoch_context,  max_subplots_per_page=200) # 10

In [ ]:
_out_pagination_controller_S = DecodedEpochSlicesPaginatedFigureController.init_from_decoder_data(short_results_obj.active_filter_epochs, short_results_obj.all_included_filter_epochs_decoder_result, included_epoch_indicies=good_epoch_indicies_S, 
	xbin=short_results_obj.original_1D_decoder.xbin, global_pos_df=global_session.position.df, a_name='Short SubsetEpochs', active_context=short_epoch_context,  max_subplots_per_page=200) # 10

In [ ]:
_out = curr_active_pipeline.display('_display_long_and_short_stacked_epoch_slices')


In [ ]:
## Getting selections out of DecoedEpochSlicesPaginationController
curr_active_pipeline.display_output_history_list
# Ideally could search like: '_display_long_and_short_stacked_epoch_slices'
active_result_backup = curr_active_pipeline.last_added_display_output

In [ ]:
list(active_result_backup.keys()) # ['name', 'context', 'figures', 'axes', 'plot_data']

active_result_backup['context'] # (IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'DecodedEpochSlices', 'replays', 'long_results_obj')>,
								#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'DecodedEpochSlices', 'replays', 'short_results_obj')>)

pagination_controllers = active_result_backup.plot_data['controllers']
long_controller = pagination_controllers[0]
short_controller = pagination_controllers[1]

In [ ]:
long_controller.selected_indicies # np.array([ 13,  14,  15,  25,  27,  28,  31,  37,  42,  45,  48,  57,  61,  62,  63,  76,  79,  82,  89,  90, 111, 112, 113, 115])

# np.array([5,  13,  15,  17,  20,  21,  24,  31,  33,  43,  44,  49,  63, 64,  66,  68,  70,  71,  74,  76,  77,  78,  84,  90,  94,  95, 104, 105, 122, 123])

In [ ]:
short_controller.selected_indicies # np.array([  9,  11,  13,  14,  15,  20,  22,  25,  37,  40,  45,  48,  61, 62,  76,  79,  84,  89,  90,  93,  94, 111, 112, 113, 115, 121])

# np.array([ 12,  13,  15,  17,  20,  24,  30,  31,  32,  33,  41,  43,  49, 54,  55,  68,  70,  71,  73,  76,  77,  78,  84,  89,  94, 100, 104, 105, 111, 114, 115, 117, 118, 122, 123, 131])

In [ ]:
final_context_L = curr_active_pipeline.build_display_context_for_session(display_fn_name='DecodedEpochSlices', epochs='replays', decoder='long_results_obj')
final_context_S = curr_active_pipeline.build_display_context_for_session(display_fn_name='DecodedEpochSlices', epochs='replays', decoder='short_results_obj')
# _out_pagination_controller.params.active_identifying_figure_ctx.adding_context(None,  user_annotation="selections")
selections_context_L = final_context_L.adding_context(None,  user_annotation="selections")
selections_context_S = final_context_S.adding_context(None,  user_annotation="selections")

# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19',display_fn_name='DecodedEpochSlices',epochs='replays',decoder='long_results_obj',user_annotation='selections')
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19',display_fn_name='DecodedEpochSlices',epochs='replays',decoder='short_results_obj')

In [ ]:
# Update the `long_results_obj.active_filter_epochs` and `short_results_obj.active_filter_epochs` objects with the selection information
assert long_controller.is_selected.shape[0] == long_results_obj.active_filter_epochs.n_epochs
assert short_controller.is_selected.shape[0] == short_results_obj.active_filter_epochs.n_epochs
# for some reason the active_filter_epochs on both the long and the short objects contain properties about the other one as well, so keep with tradition and add them to both as well
long_results_obj.active_filter_epochs._df['long_is_user_included'] = long_controller.is_selected
long_results_obj.active_filter_epochs._df['short_is_user_included'] = short_controller.is_selected
short_results_obj.active_filter_epochs._df['long_is_user_included'] = long_controller.is_selected
short_results_obj.active_filter_epochs._df['short_is_user_included'] = short_controller.is_selected

In [ ]:
print(list(long_controller.params.keys())) # ['name', 'window_title', 'num_slices', '_debug_test_max_num_slices', 'active_num_slices', 'global_epoch_start_t', 'global_epoch_end_t', 'single_plot_fixed_height', 'all_plots_height', 'epoch_labels', 'variable_name', 'xbin', 'enable_flat_line_drawing', 'debug_print', 'active_identifying_figure_ctx', 'flat_all_data_indicies', 'is_selected', 'callback_id', 'on_render_page_callbacks']

# TODO 2023-06-19 14:31: - [ ] Come up with system for storing user-annotations:
 Need to store user-annotations and labeled selections for Epochs:

	date_annotated: datetime - the date the annotations were made
	annotation_author: str - the person who made the annotations
	annotation_content: Any - the content of the annotation



In [ ]:
from PendingNotebookCode import get_user_annotations

# Get the manual user annotations to determine the good replays for both long/short decoding:
user_annotations = get_user_annotations()

final_context_L = curr_active_pipeline.build_display_context_for_session(display_fn_name='DecodedEpochSlices', epochs='replays', decoder='long_results_obj')
final_context_S = curr_active_pipeline.build_display_context_for_session(display_fn_name='DecodedEpochSlices', epochs='replays', decoder='short_results_obj')
# _out_pagination_controller.params.active_identifying_figure_ctx.adding_context(None,  user_annotation="selections")
selections_context_L = final_context_L.adding_context(None,  user_annotation="selections")
selections_context_S = final_context_S.adding_context(None,  user_annotation="selections")

## try to get the user annotations for this session:
selection_idxs_L = user_annotations[selections_context_L]
selection_idxs_S = user_annotations[selections_context_S]

selection_idxs_L

In [ ]:
selection_idxs_S

In [ ]:
# For updating the controller from the selected indicies:
for a_selected_index in selection_idxs_L:
	_out_pagination_controller.params.is_selected[a_selected_index] = True

_out_pagination_controller.perform_update_selections(defer_render=False)

# Other Programmatic Figures

In [ ]:
curr_active_pipeline.reload_default_display_functions()

In [ ]:
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import _find_any_context_neurons

## Builds proper sort indicies for '_display_short_long_pf1D_comparison'
long_ratemap = long_pf1D.ratemap
short_ratemap = short_pf1D.ratemap

curr_any_context_neurons = _find_any_context_neurons(*[k.neuron_ids for k in [long_ratemap, short_ratemap]])
curr_any_context_neurons

In [ ]:
from neuropy.plotting.ratemaps import plot_ratemap_1D
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import plot_short_v_long_pf1D_comparison, LongShortTrackComparingDisplayFunctions

curr_active_pipeline.reload_default_display_functions()

%matplotlib qt
active_identifying_session_ctx = curr_active_pipeline.sess.get_context() # 'bapun_RatN_Day4_2019-10-15_11-30-06'

long_single_cell_pfmap_processing_fn = None
short_single_cell_pfmap_processing_fn = None
# sort_idx = None
# sort_idx = sort_ind
# sort_idx = sort_indicies
# sort_idx = new_all_aclus_sort_indicies
sort_idx = 'peak_long'
# sort_idx = 'bad'

# out = curr_active_pipeline.display('_display_short_long_pf1D_comparison', active_identifying_session_ctx, single_figure=False, debug_print=True, fignum='Short v Long pf1D Comparison',
#                                    long_kwargs={'included_unit_neuron_IDs': curr_any_context_neurons, 'included_unit_indicies': None, 'sortby': sort_idx, 'single_cell_pfmap_processing_fn': long_single_cell_pfmap_processing_fn},
#                                    short_kwargs={'included_unit_neuron_IDs': curr_any_context_neurons, 'included_unit_indicies': None,'sortby': sort_idx, 'single_cell_pfmap_processing_fn': short_single_cell_pfmap_processing_fn, 'curve_hatch_style': {'hatch':'///', 'edgecolor':'k'}},
#                                   )

out = curr_active_pipeline.display('_display_short_long_pf1D_comparison', active_identifying_session_ctx, single_figure=False, debug_print=False, fignum='Short v Long pf1D Comparison',
                                   long_kwargs={'sortby': sort_idx, 'single_cell_pfmap_processing_fn': long_single_cell_pfmap_processing_fn},
                                   short_kwargs={'sortby': sort_idx, 'single_cell_pfmap_processing_fn': short_single_cell_pfmap_processing_fn, 'curve_hatch_style': {'hatch':'///', 'edgecolor':'k'}},
                                  )
                                  

# ax = out.axes[0]

In [ ]:
curr_active_pipeline.display('_display_short_long_pf1D_comparison', single_figure=True)

# 2023-06-01 - Testing new metrics:

In [ ]:

curr_long_short_post_decoding = curr_active_pipeline.global_computation_results.computed_data['long_short_post_decoding']
## Extract variables from results object:
expected_v_observed_result, curr_long_short_rr = curr_long_short_post_decoding.expected_v_observed_result, curr_long_short_post_decoding.rate_remapping
rate_remapping_df, high_remapping_cells_only = curr_long_short_rr.rr_df, curr_long_short_rr.high_only_rr_df
Flat_epoch_time_bins_mean, Flat_decoder_time_bin_centers, num_neurons, num_timebins_in_epoch, num_total_flat_timebins, is_short_track_epoch, is_long_track_epoch, short_short_diff, long_long_diff = expected_v_observed_result['Flat_epoch_time_bins_mean'], expected_v_observed_result['Flat_decoder_time_bin_centers'], expected_v_observed_result['num_neurons'], expected_v_observed_result['num_timebins_in_epoch'], expected_v_observed_result['num_total_flat_timebins'], expected_v_observed_result['is_short_track_epoch'], expected_v_observed_result['is_long_track_epoch'], expected_v_observed_result['short_short_diff'], expected_v_observed_result['long_long_diff']

print(f'long_test: {np.mean(long_long_diff)}')
print(f'short_test: {np.mean(short_short_diff)}')

In [ ]:
# long_results_obj.original_1D_decoder
replay_result_df = deepcopy(long_results_obj.active_filter_epochs.to_dataframe())
replay_result_df

In [ ]:
short_results_obj.active_filter_epochs.to_dataframe()

In [ ]:
# Look at lap speed over time - doesn't look like there's a difference between the long and the short track speeds
global_session.position.compute_higher_order_derivatives()
running_pos_df = global_session.position.to_dataframe()
ax = running_pos_df.plot(x='t', y=['lin_pos','speed'], title='Running Speed over Time')
ax

In [ ]:
# Call the function
fig, ax = plot_lines(replay_result_df)

# 2023-06-13 11:01: - [ ] New display function testing

In [ ]:
curr_active_pipeline.reload_default_display_functions()


In [ ]:

jonathan_firing_rate_analysis_result = JonathanFiringRateAnalysisResult(**curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis.to_dict())
neuron_replay_stats_df, short_exclusive, long_exclusive, BOTH_subset, EITHER_subset, XOR_subset, NEITHER_subset = jonathan_firing_rate_analysis_result.get_cell_track_partitions()

# included_neuron_ids = deepcopy(exclusive_aclus)
# included_neuron_ids = None
included_neuron_ids = EITHER_subset.track_exclusive_aclus
spikes_df: pd.DataFrame = deepcopy(curr_active_pipeline.sess.spikes_df).spikes.sliced_by_neuron_type('pyr')
# filter_epochs = deepcopy(curr_active_pipeline.sess.replay)
# spikes_df = deepcopy(long_results_obj.spikes_df) # LeaveOneOutDecodingAnalysisResult
# spikes_df[np.isin(spikes_df.aclu, included_neuron_ids)]
filter_epochs_df = deepcopy(long_results_obj.active_filter_epochs.to_dataframe())


In [ ]:
app, win, plots, plots_data = plot_multiple_raster_plot(considered_filter_epochs_df, filter_epoch_spikes_df, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=new_all_aclus_sort_indicies, epoch_id_key_name='replay_epoch_id', scatter_app_name="Pho Custom Selected Stacked Replays")

In [ ]:
app, win, plots, plots_data = plot_multiple_raster_plot(active_filter_epochs, filter_epoch_spikes_df, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=new_all_aclus_sort_indicies, epoch_id_key_name='replay_epoch_id', scatter_app_name="Selected Stacked Replays - Short Side")

In [ ]:
app, win, plots, plots_data = plot_multiple_raster_plot(active_filter_epochs, filter_epoch_spikes_df, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=new_all_aclus_sort_indicies, epoch_id_key_name='replay_epoch_id', scatter_app_name="Selected Stacked Replays - LONG Side")

In [ ]:
for epoch_idx, new_ax in plots.ax.items():
	new_ax.getAxis('bottom').setTickSpacing(None) # 5.0, 1.0 .setTickSpacing(x=[None], y=[1.0])
	new_ax.showGrid(x=False, y=False, alpha=1.0)
	new_ax.getAxis('bottom').setStyle(showValues=False)


In [ ]:
considered_short_side_epochs_df = filter_epochs_df[ (filter_epochs_df['has_SHORT_exclusive_aclu'] & filter_epochs_df['short_is_user_included'] & ~filter_epochs_df['long_is_user_included']) ]  # replay not considered good by user for long decoding, but it is for short decoding. Finally, has at least one SHORT exclusive ACLU.
considered_short_side_epochs_df

considered_long_side_epochs_df = filter_epochs_df[ (filter_epochs_df['has_LONG_exclusive_aclu'] & ~filter_epochs_df['short_is_user_included'] & filter_epochs_df['long_is_user_included']) ] # replay not considered good by user for short decoding, but it is for long decoding. Finally, has at least one LONG exclusive ACLU.
considered_long_side_epochs_df